In [ ]:
!pip install datasets
!pip install transformers

from datasets import load_dataset
import pandas as pd


dataset = load_dataset("webis/args_me")



# print(dataset.keys())


split_key = 'train'
df = pd.DataFrame(dataset[split_key])
# display(df)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.1
    Uninstalling huggingface-hub-0.19.1:
      Successfully uninstalled huggingface-hub-0.19.1


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
df = df[:250]
print(df)

                                              argument  \
0    My opponent forfeited every round. None of my ...   
1    How do you propose the school will fund your p...   
2    Schools have no compelling interest in providi...   
3    As a senior at my school. My group and I are f...   
4    The resolution used by Pro *assumes* that Aust...   
..                                                 ...   
245  BOP- My opponent has it as she advocates a cha...   
246  I will be defending the fact that pornography ...   
247               I can devils advocate this I accept.   
248  My position is pretty obvious. I will prove th...   
249  Yes, you have the BOP, and I have fulfilled it...   

                                         conclusion stance  \
0      Contraceptive Forms for High School Students    CON   
1      Contraceptive Forms for High School Students    CON   
2      Contraceptive Forms for High School Students    CON   
3      Contraceptive Forms for High School Students    

In [ ]:
cn=pr=0

for i in range(len(df)):
  if(df.iloc[i]['stance']=='CON'):
    cn+=1
  if(df.iloc[i]['stance']=='PRO'):
    pr+=1

# print(cn,pr)

In [ ]:
dic={}
for i in range(len(df)):
  if((df.iloc[i]['conclusion'],df.iloc[i]['stance']) not in dic):
    dic[(df.iloc[i]['conclusion'],df.iloc[i]['stance'])] = [df.iloc[i]['argument']]
  else:
    dic[(df.iloc[i]['conclusion'],df.iloc[i]['stance'])].append(df.iloc[i]['argument'])

In [ ]:
for i in range(len(df)):
  if((df.iloc[i]['conclusion'],'CON') in dic or (df.iloc[i]['conclusion'],'PRO') in dic):
    if((df.iloc[i]['conclusion'],'CON') in dic and len(dic[(df.iloc[i]['conclusion'],'CON')])<2):
      dic.pop((df.iloc[i]['conclusion'],'CON'),None)
      dic.pop((df.iloc[i]['conclusion'],'PRO'),None)
    if((df.iloc[i]['conclusion'],'PRO') in dic and len(dic[(df.iloc[i]['conclusion'],'PRO')])<2):
      dic.pop((df.iloc[i]['conclusion'],'PRO'),None)
      dic.pop((df.iloc[i]['conclusion'],'CON'),None)


print(len(dic))

64


In [ ]:
# queries = set()

# for i in dic:
#   queries.add(i[0])

In [ ]:
print(queries)
print(len(queries))

{'Dance Is Not a Sport', 'Islam is a religion of peace', 'The closet dementia of the superior ego god complex, The bible and why you should not believe in god', 'Football is for women as well as men.', 'My Proposal for the Eradication of Extreme Poverty', 'Baseball is a better sport than basketball.', 'Russell Hantz Should Have Won Survivor: Samoa', 'School Uniforms', 'Laws against voluntary polygamous marriages are just', 'The system of "Sin" is not fair.', 'The question of rape in the bride kidnapping video', 'pornography ought to be illegal', "Anybody who used performance enhancing drugs in the MLB shouldn't be elected to the Hall of fame", 'People should not text and drive.', 'guitar is better than rock band', 'Rap Battle', 'Debates are harder when you are the first one to come up with points', 'The Universe Was Created By God', 'Global warming is a bigger issue than World Peace', 'Resolved: The Ontological Argument is Sound', 'life', "In Western Countries, Men's rights are more of

In [ ]:
from transformers import pipeline, BertTokenizer, BertForMaskedLM

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Define a function to predict masked words


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def predict_masked_words(sentence):
    # Tokenize the input sentence
    tokens = tokenizer(sentence, return_tensors="pt")

    # Get the position of the masked token
    masked_index = tokens["input_ids"][0].tolist().index(tokenizer.mask_token_id)

    # Get the logits for the masked token
    logits = model(**tokens).logits

    # Get the predictions for the masked token
    predictions = logits[0, masked_index].topk(5).indices.tolist()

    # Convert predictions back to words
    predicted_words = [tokenizer.decode([prediction]) for prediction in predictions]

    return predicted_words[:5]

# Example usage
sentence = "The quick brown [MASK] jumps over the lazy dog."
predicted_words = predict_masked_words(sentence)

print(f"Original sentence: {sentence}")
print(f"Predicted words: {predicted_words}")

Original sentence: The quick brown [MASK] jumps over the lazy dog.
Predicted words: ['cat', 'dog', 'bear', 'man', '##ie']


In [ ]:
!pip install nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
#pre-processing

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources
nltk.download("stopwords")
nltk.download("punkt")

# Define preprocessing functions
def preprocess_text(text):

    tokens = word_tokenize(text)
    #print("after word tokenise :- ")
    # print(len(tokens))

    # Stopword removal
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    #print("after stopwords removal :- ")
    #print(len(tokens))



    # Join tokens back into text
    preprocessed_text = " ".join(tokens)

    return preprocessed_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def expand(text):
  clean_text = preprocess_text(text)
  # print(clean_text)
  temp = clean_text.split(" ")
  ans = ""
  for i in range(len(temp)):
    ans = ans + ' ' + temp[i] +' ' + ' '.join(predict_masked_words(' '.join(temp[:i]) + ' [MASK] ' + ' '.join(temp[i+1:])))
    ans+=' '

  return ans
# print(expand('I love apples'))

In [ ]:
# for i in dic:
#   print(expand(i[0]))

In [ ]:
exp_queries = {}

for i in dic:
  if(i[1]=='PRO'):
    temp = 'I Think '+i[0]+''' is good because of it's benifits'''
  else:
    temp = 'No, I think ' + i[0] + 'is bad'
  exp_queries[(i[0],i[1])] = expand(temp)
print(exp_queries)

{('Australia should be a more significant country', 'CON'): ' , i they we you people  think in see south and western  Australia of a about , your  significant is ##is , . ##s  countryis and is or , -  bad ? . ; ! | ', ('Australia should be a more significant country', 'PRO'): " Think south western in north -  Australia of about a , that  significant the a is good ,  country and for the a australia  good australia it women people who  's country people nation for world  benifits . ! ; ? | ", ('The closet dementia of the superior ego god complex, The bible and why you should not believe in god', 'CON'): ' , i we they you bible  think the a in and bible  closet of about , no the  dementia ##ed ##ly , of a  superior , and / . :  ego , to - . of  god ##s ##ism ##is - ##istic  complex ##is ##i ##us . ##es  , the , . in and  bible and or , i so  believe god , bad - of  godis no it nothing not me  bad . ! ? ; | ', ('The closet dementia of the superior ego god complex, The bible and why you sho

In [ ]:
print(exp_queries)

{('Australia should be a more significant country', 'CON'): ' , i they we you people  think in see south and western  Australia of a about , your  significant is ##is , . ##s  countryis and is or , -  bad ? . ; ! | ', ('Australia should be a more significant country', 'PRO'): " Think south western in north -  Australia of about a , that  significant the a is good ,  country and for the a australia  good australia it women people who  's country people nation for world  benifits . ! ; ? | ", ('The closet dementia of the superior ego god complex, The bible and why you should not believe in god', 'CON'): ' , i we they you bible  think the a in and bible  closet of about , no the  dementia ##ed ##ly , of a  superior , and / . :  ego , to - . of  god ##s ##ism ##is - ##istic  complex ##is ##i ##us . ##es  , the , . in and  bible and or , i so  believe god , bad - of  godis no it nothing not me  bad . ! ? ; | ', ('The closet dementia of the superior ego god complex, The bible and why you sho

In [ ]:
print(len(exp_queries))

138


In [ ]:
docs=[]
for i in dic:
  docs+=dic[i]

In [ ]:
print(len(docs))

199


In [ ]:
print(docs)

["The resolution used by Pro *assumes* that Australia isn't already a 'significant' country - however, in actual reality, it is. Firstly we should clarify what significance means: 1.a the state or quality of being significant1.b of consequence or importance==================================To respond directly to Pros argument first:he/she asserts that Australia invented 'amazing things' like 'WiFI, Google Maps, Polymer bank notes, Ultrasound scanners, stainless steel braces and many more things'. Now, if these inventions did come from Australia, then, it can be considered a 'significant country' - as a country which is the home of some of the most universally-used inventions in the 21st century. It would seem that Pro was/is trying to argue that Australia simply deserves more recognition, in which case he/she should proposed that instead of stating that it should be more 'significant' - because the examples that Pro themselves has listed, fully go against that. Instead of affirming the

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:

def cos_sim(text1,text2):
  vectorizer = TfidfVectorizer()

  # Transform the texts into TF-IDF vectors
  tfidf_matrix = vectorizer.fit_transform([text1, text2])

  # Calculate the cosine similarity
  cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

  return cosine_sim

In [ ]:
print(docs)
for i in dic:
  print(dic[i])

["The resolution used by Pro *assumes* that Australia isn't already a 'significant' country - however, in actual reality, it is. Firstly we should clarify what significance means: 1.a the state or quality of being significant1.b of consequence or importance==================================To respond directly to Pros argument first:he/she asserts that Australia invented 'amazing things' like 'WiFI, Google Maps, Polymer bank notes, Ultrasound scanners, stainless steel braces and many more things'. Now, if these inventions did come from Australia, then, it can be considered a 'significant country' - as a country which is the home of some of the most universally-used inventions in the 21st century. It would seem that Pro was/is trying to argue that Australia simply deserves more recognition, in which case he/she should proposed that instead of stating that it should be more 'significant' - because the examples that Pro themselves has listed, fully go against that. Instead of affirming the

In [ ]:
c=0
ans = {}
for i in dic:
  temp = []
  for j in docs:
    # j=preprocess_text(j)
    temp.append((j,cos_sim(exp_queries[i],preprocess_text(j))[0][1]))
  # print(temp[0][1])
  temp.sort(key=lambda x:x[1],reverse=True)
  temp = temp[:3]
  ans[i] = temp
  for k in temp:
    if(k[0] in dic[i]):
      c+=1
      break
print(c,len(dic))

74 138


In [ ]:
print(len(ans))

138


In [ ]:
# for i in dic:
#   for j in dic[i]:
#     print(cos_sim(exp_queries[i],(j))[0][1])

In [ ]:
!pip install tensorflow tensorflow-hub

import tensorflow as tf
import tensorflow_hub as hub

# Load the Universal Sentence Encoder model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)



In [ ]:
import numpy as np
def embed_document(document,query):
    # Convert the document to a list of strings
    sentences = [document]

    # Generate embeddings
    document_embedding = model(sentences)
    query_embedding = model([query])

    document_embedding_np = document_embedding.numpy()
    embed_np = query_embedding.numpy()

    return(np.linalg.norm(document_embedding - query_embedding))

    # Return the 512-dimensional embedding
    # return embeddings



# print(type(document_embedding))

# The resulting document embedding is a TensorFlow Tensor
# You can convert it to a NumPy array for easier handling if needed

# # Print the shape of the resulting embedding
# print(f"Document embedding shape: {document_embedding_np.shape}")
# print(document_embedding)

In [ ]:
#USE
use_queries = {}

In [ ]:
c=0
for i in dic:
  temp=''
  if(i[1]=='PRO'):
    temp = 'I Think '+i[0]+''' is good because of it's benifits'''
  else:
    temp = 'No, I think ' + i[0] + 'is bad'
  # temp = embed_document(temp)
  ans =[]
  for j in docs:
    ans.append([j,embed_document(temp,j)])
  ans.sort(key=lambda x:x[1])
  for k in range(5):
    if(ans[k][0] in dic[i]):
      c+=1
      break

print(c,len(dic))

57 64


In [ ]:
import pandas as pd
ddff = pd.read_csv('20queries.csv')


FileNotFoundError: ignored

In [ ]:
# GPT


query=[]
document = []

for i in range(len(ddff)):
  if(df.iloc[i]['query'] not in query):
    query.append(df.iloc[i]['query'])
  document.append(df.iloc[i]['document'])


In [ ]:





for i in query:
  mx=0
  temp=""
  for j in document:
    if(cos_sim(i,j)>mx):
      mx=cos_sim(i,j)
      temp=j
  print(i,j)







In [ ]:
# prompt: implement k means clustering

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load the data
data = pd.read_csv('data.csv')

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the StandardScaler object to the data
scaler.fit(data)

# Transform the data using the StandardScaler object
scaled_data = scaler.transform(data)

# Create a KMeans object with 3 clusters
kmeans = KMeans(n_clusters=3)

# Fit the KMeans object to the scaled data
kmeans.fit(scaled_data)

# Get the cluster labels for each data point
labels = kmeans.labels_

# Print the cluster labels
print(labels)
